In [1]:
from run_capsule import *
from decoding_utils import *
import sys
sys.argv[1:] = ['--result_prefix', 'debug']
params = Params()

In [30]:
current = Params.model_validate_json(
    upath.UPath('s3://aind-scratch-data/dynamic-routing/ben/decoding/debug_20250327_165658/params.json').read_text()
)
current

Params(result_prefix='debug', session_id='703333_2024-04-10', run_id='20250327_165658', skip_existing=False, test=False, logging_level='INFO', update_packages_from_source=False, override_params_json='{}', session_table_query="is_ephys & is_task & is_annotated & is_production & issues=='[]'", unit_criteria='medium', n_units=20, n_repeats=25, input_data_type='spikes', vid_angle_facemotion='face', vid_angle_LP='behavior', central_section='4_blocks_plus', exclude_cue_trials=False, n_unit_threshold=5, keep_n_SVDs=500, LP_parts_to_keep=['ear_base_l', 'eye_bottom_l', 'jaw', 'nose_tip', 'whisker_pad_l_side'], spikes_binsize=0.2, spikes_time_before=0.2, spikes_time_after=0.01, use_structure_probe=True, crossval='5_fold', labels_as_index=True, decoder_type='LogisticRegression', only_use_all_units=False, predict='context', regularization=None, penalty=None, solver=None, select_single_area=None, split_area_by_probe=True, n_jobs=None, use_process_pool=True, max_workers=None)

In [38]:
new = Params(
    result_prefix='debug', session_id='703333_2024-04-10', run_id='20250327_165658', override_params_json='{}', skip_existing=True,
)
new.model_dump() == current.model_dump()

False

In [4]:
delta = pl.scan_delta(
    's3://aind-scratch-data/dynamic-routing/ben/decoding/debug_20250327_165658'
)

In [19]:
(
    delta
    .group_by('session_id')
    .agg(
        pl.col('structure').n_unique().alias('n_structures'),
    )
    .collect()
    .sort('n_structures')
)

session_id,n_structures
str,u32
"""662892_2023-08-21""",1
"""702136_2024-03-04""",1
"""668755_2023-08-31""",1
"""702136_2024-03-06""",1
"""712815_2024-05-22""",1
…,…
"""703880_2024-04-18""",7
"""733891_2024-09-18""",7
"""712815_2024-05-21""",7


In [12]:
(
    delta
    .sort('repeat_idx')
    # for each repeat, get mean accuracy across all shifts
    .group_by('session_id', 'structure', 'electrode_group_name', 'n_units', 'repeat_idx', maintain_order=True)
    .agg(
        pl.mean('balanced_accuracy_test').alias('mean_balanced_accuracy_test'),
        pl.col('balanced_accuracy_test').filter(pl.col('shift_idx').eq(0)).first().alias('balanced_accuracy_test_shift_0'),
    )
    # get mean of 0-shifts across repeats
    .group_by('session_id', 'structure', 'electrode_group_name', 'n_units')
    .agg(
        pl.all().exclude('balanced_accuracy_test_shift_0'),
        pl.mean('balanced_accuracy_test_shift_0').alias('mean_balanced_accuracy_test_shift_0'),
    )
    .collect()
)

session_id,structure,electrode_group_name,n_units,repeat_idx,mean_balanced_accuracy_test,mean_balanced_accuracy_test_shift_0
str,str,str,i32,list[i64],list[f64],f64
"""649943_2023-02-16""","""MGv""","""probeD""",20,"[0, 1, … 24]","[0.547665, 0.539835, … 0.540562]",0.585301
"""741137_2024-10-11""","""SCm""","""probeC""",20,"[0, 1, … 24]","[0.684017, 0.685999, … 0.659854]",0.765812
"""636766_2023-01-26""","""ILA""","""probeE""",20,"[0, 1, … 24]","[0.541578, 0.557304, … 0.56375]",0.591123
"""726088_2024-06-20""","""DP""","""probeF""",20,"[0, 1, … 24]","[0.620756, 0.621033, … 0.614592]",0.665033
"""702136_2024-03-05""","""LD""","""probeB""",20,"[0, 1, … 24]","[0.513912, 0.540432, … 0.53868]",0.56173
…,…,…,…,…,…,…
"""712815_2024-05-21""","""SMT""","""probeA""",20,"[0, 1, … 24]","[0.567256, 0.576199, … 0.581139]",0.638409
"""741137_2024-10-11""","""POST""","""probeD""",20,[0],[0.555036],0.574621
"""702131_2024-02-26""","""SSp""","""probeA""",20,"[0, 1, … 24]","[0.525138, 0.556091, … 0.537326]",0.583978


In [14]:
units = get_filtered_units(params).collect()

In [18]:
units = utils.get_df('units', lazy=False)

In [21]:
params.units_group_by

[<Expr ['col("session_id")'] at 0x7EFC096D8B80>,
 <Expr ['col("structure")'] at 0x7EFC096D9F90>,
 <Expr ['col("electrode_group_name")'] at 0x7EFC096D9720>]

In [31]:
params.units_query

<Expr ['[([([(col("isi_violations_rati…'] at 0x7EFC096DA080>

In [33]:
(
    units
    .pipe(group_structures, keep_originals=True)
    .filter(
        params.units_query,
        pl.col('isi_violations_ratio').le(0.5),
        pl.col('presence_ratio').ge(0.9),
        pl.col('amplitude_cutoff').le
    )
    .filter(
        pl.col('unit_id').n_unique().ge(params.n_units).over(params.units_group_by),
    )
    .group_by(*params.units_group_by)
    .agg(
        pl.col('unit_id').n_unique().alias('n_units'),
    )
    .sort('n_units')
)


session_id,structure,electrode_group_name,n_units
str,str,str,u32
"""703333_2024-04-10""","""out of brain""","""probeB""",1
"""644864_2023-01-31""","""out of brain""","""probeA""",1
"""699847_2024-04-16""","""out of brain""","""probeB""",1
"""636397_2022-09-27""","""out of brain""","""probeF""",1
"""626791_2022-08-16""","""out of brain""","""probeC""",1
…,…,…,…
"""746439_2025-01-29""",null,"""probeE""",537
"""670180_2023-07-25""","""ProS""","""probeC""",552
"""744740_2024-11-13""",null,"""probeE""",584


In [17]:
units.group_by(*params.units_group_by).agg(pl.col('unit_id').n_unique().alias('n_units')).sort('n_units')

session_id,structure,electrode_group_name,n_units
str,str,str,u32
"""626791_2022-08-16""","""out of brain""","""probeC""",1
"""742903_2024-10-23""","""DG""","""probeE""",1
"""733780_2024-08-26""","""TTd""","""probeF""",1
"""699847_2024-04-16""","""out of brain""","""probeB""",1
"""636766_2023-01-25""","""out of brain""","""probeE""",1
…,…,…,…
"""746439_2025-01-29""",null,"""probeE""",537
"""670180_2023-07-25""","""ProS""","""probeC""",552
"""744740_2024-11-13""",null,"""probeE""",584
